In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import cv2
import os

In [22]:
class dsprite_color(Dataset):
    def __init__(self,balance_matrix:"3x3 ndarray"):
        """
        color * shape
        rc rh rs
        gc gh gs
        bc bh bs
        """
        if balance_matrix.shape[0] != 3 or balance_matrix.shape[1] != 3:
            raise "error balance_matrix out of range should be 3*3"
        
        self.imgs_path = "dsprite_color_fig/"
        self.colors=['red/','green/','blue/']
        self.shapes=['circle/','heart/','square/']
        
        self.data = []
        for color_i in range(balance_matrix.shape[0]):
            for shape_i in range(balance_matrix.shape[1]):
                folder_path=self.imgs_path+self.shapes[shape_i]+self.colors[color_i]
                img_paths=os.listdir(folder_path)
                # load only data_numbers data
                data_number=int(balance_matrix[color_i,shape_i]*len(img_paths))
                for i in range(data_number):
                    self.data.append([folder_path+img_paths[i] ,shape_i,color_i])
        self.color_map = {0:"red",1:"green",2:"blue"}
        self.shape_map ={0:"circle",1:"heart",2:"square"}
        self.img_dim = (64,64)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        img_path , shape_id , color_id = self.data[idx]
        img = cv2.imread(img_path)
        img = cv2.resize(img, self.img_dim)
        img_tensor = torch.from_numpy(img)
        img_tensor = img_tensor.permute(2, 0, 1)
        shape_id = torch.tensor([shape_id])
        color_id = torch.tensor([color_id])
        return img_tensor, shape_id, color_id 

In [23]:
data_loader=dsprite_color(np.arange(9).reshape(3,3)/8)

In [25]:
inputs, shape,color = next(iter(data_loader))  